In [ ]:
import pandas as pd
import requests
import bs4
from time import sleep

In [ ]:
months = []
months_range = range(1, 13)
for month_num in months_range:
    req = requests.get(
        f"https://www.weathertab.com/en/c/e/{month_num}/united-states/kansas/topeka/")
    month_soup = bs4.BeautifulSoup(req.text)
    day_svg = month_soup.find('div', class_='calendarNew').find_all('svg')
    for d in day_svg:
        day_text = d.find('a').find_all('text')
        day_forecast = {}
        for day in day_text:
            day_forecast['month'] = f'{month_num}'
            for the_day in day_text[:4]:    #the last two elements are just the temp in C but has the same class name, so just ignore them
                if the_day['class'][0] == 'dt':
                    day_forecast['day'] = the_day.text
                if the_day['class'][0] == 'pct':
                    day_forecast['precip'] = the_day.text   
                if the_day['class'][0] == 'hot' :
                    day_forecast['high'] = the_day.text  
                if the_day['class'][0] == 'cold' :
                    day_forecast['cold'] = the_day.text  
        months.append(day_forecast)

In [ ]:
#check the results
months_bk = months.copy() #why not make a back up   ¯\_(ツ)_/¯
pd.DataFrame(months_bk).sample(10)

In [ ]:
#optional, get more details for each day
for day in months:
    print(f"getting month {day['month']} , day {day['day']}")
    day_url = f"https://www.weathertab.com/en/long-range-weather/e/{day['month']}/{day['day']}/united-states/kansas/topeka/?ref=c"
    day_soup = bs4.BeautifulSoup(requests.get(day_url).text)
    day['forecast'] = day_soup.find('td',class_='fct_detail_img').find('svg')['title']
    for t in day_soup.find_all('div', class_='F'):
        cleaned_text = t.text.strip().split(' ')
        if (t.text.find('High Temperature Forecast')):
            day['high_temp_low'] = cleaned_text[3].strip()
            day['high_temp_high'] = cleaned_text[5].strip()
        else:
            day['low_temp_low'] = cleaned_text[3].strip()
            day['low_temp_high'] = cleaned_text[5].strip()

In [ ]:
weather_df = pd.DataFrame(months).drop_duplicates()

In [ ]:
#save it
weather_df.to_csv("2024_forecast.csv", index=False)

In [ ]:
#checkt the saved file
pd.read_csv("2024_forecast.csv")